In [2]:
import os

# 현재 작업 디렉토리 출력
current_directory = os.getcwd()
print("현재 작업 디렉토리:", current_directory)

# 디렉토리 변경
new_directory = 'C:/Users/bimba/datamining/project/DATA'  # 원하는 디렉토리 경로로 변경
os.chdir(new_directory)

# 변경된 작업 디렉토리 출력
updated_directory = os.getcwd()
print("변경된 작업 디렉토리:", updated_directory)

import pandas as pd
import folium
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# 데이터 로드
file = pd.read_csv('yeo_pm.csv', encoding='cp949')
station = pd.DataFrame(file)

# Folium 지도
initial_location = [37.525989, 126.923948]
my_map = folium.Map(location=initial_location, zoom_start=14)
for index, row in station.iterrows():
    lat, lon = row['X'], row['Y']
    folium.Marker(location=[lat, lon], popup=row['대여소명']).add_to(my_map)


현재 작업 디렉토리: C:\Users\bimba\datamining\project\PM_OD\1_6
변경된 작업 디렉토리: C:\Users\bimba\datamining\project\DATA


In [6]:
file_legend_mappings = {
    '1월 시간대별 대여현황': {
        'OD_230101.csv': '01/01',
        'OD_230102.csv': '01/02',
        'OD_230103.csv': '01/03',
        'OD_230104.csv': '01/04',
        'OD_230105.csv': '01/05',
        'OD_230106.csv': '01/06',
        'OD_230107.csv': '01/07'
    },
    '2월 시간대별 대여현황': {
        'OD_230201.csv': '02/01',
        'OD_230202.csv': '02/02',
        'OD_230203.csv': '02/03',
        'OD_230204.csv': '02/04',
        'OD_230205.csv': '02/05',
        'OD_230206.csv': '02/06',
        'OD_230207.csv': '02/07'
    },
    '3월 시간대별 대여현황': {
        'OD_230301.csv': '03/01',
        'OD_230302.csv': '03/02',
        'OD_230303.csv': '03/03',
        'OD_230304.csv': '03/04',
        'OD_230305.csv': '03/05',
        'OD_230306.csv': '03/06',
        'OD_230307.csv': '03/07'
    },
    '4월 시간대별 대여현황': {
        'OD_230401.csv': '04/01',
        'OD_230402.csv': '04/02',
        'OD_230403.csv': '04/03',
        'OD_230404.csv': '04/04',
        'OD_230405.csv': '04/05',
        'OD_230406.csv': '04/06',
        'OD_230407.csv': '04/07'
    },
    '5월 시간대별 대여현황': {
        'OD_230501.csv': '05/01',
        'OD_230502.csv': '05/02',
        'OD_230503.csv': '05/03',
        'OD_230504.csv': '05/04',
        'OD_230505.csv': '05/05',
        'OD_230506.csv': '05/06',
        'OD_230507.csv': '05/07'
    },
    '6월 시간대별 대여현황': {
        'OD_230601.csv': '06/01',
        'OD_230602.csv': '06/02',
        'OD_230603.csv': '06/03',
        'OD_230604.csv': '06/04',
        'OD_230605.csv': '06/05',
        'OD_230606.csv': '06/06',
        'OD_230607.csv': '06/07'
    }
}

# Load all data
file_paths = {
    '1월 대여소별 대여현황': 'PM_CNT_1.csv',
    '2월 대여소별 대여현황': 'PM_CNT_2.csv',
    '3월 대여소별 대여현황': 'PM_CNT_3.csv',
    '4월 대여소별 대여현황': 'PM_CNT_4.csv',
    '5월 대여소별 대여현황': 'PM_CNT_5.csv',
    '6월 대여소별 대여현황': 'PM_CNT_6.csv'
}

In [9]:
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Tabs([
        # 탭 1
        dcc.Tab(label='여의도내 따릉이 대여소 위치현황', children=[
            html.Div([
                html.Iframe(id='map', srcDoc=my_map._repr_html_(), width='100%', height='600'),
            ])
        ]),

        # 탭 2
        dcc.Tab(label='시간대별 대여현황', children=[
            html.Div([
                dcc.Dropdown(
                    id='file-dropdown',
                    options=[
                        {'label': key, 'value': key} for key in file_legend_mappings.keys()
                    ],
                    value='1월 시간대별 대여현황',
                    style={'width': '50%'}
                ),
                dcc.Graph(id='line-plot')
            ])
        ]),

        # 탭 3
        dcc.Tab(label='자치구별 대여현황', children=[
            html.Div([
                html.H1("따릉이 자치구별 대여현황"),
                dcc.Dropdown(
                    id='month-dropdown',
                    options=[{'label': month, 'value': month} for month in file_paths.keys()],
                    value='1월 대여소별 대여현황',
                    style={'width': '50%'}
                ),
                dcc.Graph(id='bar-chart'),
            ])
        ]),
    ])
])

In [10]:
@app.callback(
    Output('line-plot', 'figure'),
    [Input('file-dropdown', 'value')]
)
def update_line_plot(selected_mapping):
    combined_data = pd.DataFrame()

    for filename, legend_label in file_legend_mappings[selected_mapping].items():
        data = pd.read_csv(filename, encoding='cp949')
        rental_count_by_hour = data.groupby('기준_시간대').size()
        combined_data[legend_label] = rental_count_by_hour

    fig = px.line(combined_data, x=combined_data.index, y=combined_data.columns, markers=True)

    fig.update_layout(
        title='시간대별 따릉이 대여현황',
        xaxis_title='시간대',
        yaxis_title='대여 횟수',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

    return fig

@app.callback(
    Output('bar-chart', 'figure'),
    [Input('month-dropdown', 'value')]
)
def update_chart(selected_month):
    data = pd.read_csv(file_paths[selected_month], encoding='cp949')
    
    # Group by '자치구' and sum '이용합계'
    grouped_data = data.groupby('자치구')['이용합계'].sum().reset_index()

    # Create bar chart using Plotly Express
    fig = px.bar(grouped_data, x='자치구', y='이용합계', title=f'따릉이 {selected_month}', labels={'이용합계': '이용 합계'})
    return fig



In [11]:
if __name__ == '__main__':
    app.run_server(debug=True, port = 7777)